In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/H2GTRM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

# feature importance 
def vis_feature_importances_(est, X_train):
    ftr_importances_values = est.feature_importances_
    ftr_importances = pd.Series(ftr_importances_values,index=X_train.columns  )
    ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
    plt.figure(figsize=(10,4))
    plt.title('Feature importances Top 20')
    sns.barplot(x=ftr_top20 , y = ftr_top20.index)
    plt.show()
    
# https://dacon.io/forum/401657    
def smape(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output    

## Data Load

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# train = pd.read_csv('/content/drive/MyDrive/work/input/train.csv')
# test = pd.read_csv('/content/drive/MyDrive/work/input/test.csv')
# building_info = pd.read_csv('/content/drive/MyDrive/work/input/building_info.csv')
# submission = pd.read_csv('/content/drive/MyDrive/work/input/sample_submission.csv')
# train.shape, test.shape, building_info.shape, submission.shape

train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
building_info = pd.read_csv('input/building_info.csv')
submission = pd.read_csv('input/sample_submission.csv')
train.shape, test.shape, building_info.shape, submission.shape

((204000, 10), (16800, 7), (100, 7), (16800, 2))

## Train Data Pre-Processing

In [4]:
# 빌딩 데이터 정리
## null 처리
building_info['태양광용량(kW)'] = building_info['태양광용량(kW)'].str.replace('-','0').astype(float)
building_info['ESS저장용량(kWh)'] = building_info['ESS저장용량(kWh)'].str.replace('-','0').astype(float)
building_info['PCS용량(kW)'] = building_info['PCS용량(kW)'].str.replace('-','0').astype(float)

In [5]:
# 정리된 빌딩과 merge
train = train.merge(building_info, on='건물번호')
test = test.merge(building_info, on='건물번호')

In [6]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train['month'] = train['일시'].apply(lambda x : int(x[4:6]))
train['day'] = train['일시'].apply(lambda x : int(x[6:8]))
train['time'] = train['일시'].apply(lambda x : int(x[9:11]))

test['month'] = test['일시'].apply(lambda x : int(x[4:6]))
test['day'] = test['일시'].apply(lambda x : int(x[6:8]))
test['time'] = test['일시'].apply(lambda x : int(x[9:11]))

In [7]:
# 요일
train['weekday'] =  pd.to_datetime(train['일시'].str[:8]).dt.weekday
test['weekday'] =  pd.to_datetime(test['일시'].str[:8]).dt.weekday

In [8]:
# outlier 제거 
train = train[train['전력소비량(kWh)'] > 1]

In [9]:
train = train.drop(train[train['num_date_time'].isin(['56_20220608 16'
                                                  ,'34_20220808 21'
                                                  ,'10_20220607 06','10_20220607 06'
                                                  ,'31_20220808 13','31_20220809 13', '31_20220808 14','31_20220808 13'
                                                  , '8_20220806 05', '8_20220707 08', '8_20220628 09'
                                                  ,'13_20220606 16','13_20220724 06','13_20220606 17','13_20220607 07'
                                                  ,'58_20220804 16'
                                                  ,'65_20220807 18'
                                                  ,'68_20220706 20','68_20220706 19'
                                                  ,'71_20220604 10','71_20220810 15','71_20220810 16'
                                                  , '73_20220707 12', '73_20220707 14', '73_20220707 11','73_20220707 13', '73_20220707 15', '73_20220808 13','73_20220803 14', '73_20220808 12', '73_20220803 15','73_20220808 11'
                                                 ])].index)

In [10]:
target_encoding = train.groupby(['건물유형']).agg({'전력소비량(kWh)':['min', 'mean','max']}).reset_index()
target_encoding.columns = ['건물유형', 'khw_min', 'khw_mean', 'khw_max']
train = train.merge(target_encoding, on=['건물유형'], how='left')
test = test.merge(target_encoding, on=['건물유형'], how='left')

In [11]:
train.drop( train[(train['건물번호'].isin([1,16])) & (train['month'] == 6)].index, inplace=True)

In [12]:
no_work_day = train.groupby(['건물번호', '건물유형','month', 'day']).agg({'전력소비량(kWh)':['mean','max']}).reset_index()
no_work_day.columns = ['건물번호', '건물유형','month', 'day', 'khw_mean', 'khw_max']
total_mean = no_work_day[no_work_day['건물유형'].isin(['백화점및아울렛','할인마트'])].groupby(['건물번호']).khw_mean.mean().reset_index().rename(columns={'khw_mean':'b_mean'})

no_work_day = no_work_day.merge(total_mean, on='건물번호')
no_work_day.loc[no_work_day.b_mean > no_work_day.khw_max, '백화점휴무일'] = 1

train = train.merge(no_work_day[['건물번호', '건물유형','month', 'day', '백화점휴무일']], on=['건물번호', '건물유형','month', 'day'], how='left')
train['백화점휴무일'] = train['백화점휴무일'].fillna(0).astype(int)
test['백화점휴무일'] = 0

In [13]:
train['냉방비율'] = ((train['냉방면적(m2)'] / train['연면적(m2)'])*100 ).astype(int)
test['냉방비율'] = ((test['냉방면적(m2)'] / test['연면적(m2)'])*100 ).astype(int)

train = train.drop(columns=['연면적(m2)','냉방면적(m2)'])
test = test.drop(columns=['연면적(m2)','냉방면적(m2)'])

In [14]:
#결측값을 0으로 채웁니다
train = train.fillna(0)
test = test.fillna(0)

In [15]:
# 공휴일 제외 
print(train.shape)
train['dates'] = train['일시'].apply(lambda x : int(x[:8]))
# train = train[~((train['dates'].isin([20220606,20220615])) & (train['건물유형'].isin(['공공','병원'])))]
train['holiday'] = 0
test['holiday'] = 0
train.loc[train['dates'].isin([20220606,20220615,20220815]), 'holiday'] = 1

train.drop(['dates'], axis=1, inplace=True)
train.shape

(202528, 23)


(202528, 24)

In [16]:
## https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
train['THI'] = 9/5*train['기온(C)'] - 0.55*(1-train['습도(%)']/100)*(9/5*train['습도(%)']-26)+32
test['THI'] = 9/5*test['기온(C)'] - 0.55*(1-test['습도(%)']/100)*(9/5*test['습도(%)']-26)+32

In [17]:
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])

for num in train['건물번호'].unique():
    temp = train[train['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

cdhs = np.array([])
for num in test['건물번호'].unique():
    temp = test[test['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs

In [18]:
############## 체감온도, https://www.weather.go.kr/plus/life/li_asset/HELP/basic/help_01_07.jsp
train['체감온도'] = 13.12 + 0.6215*train['기온(C)'] - 11.37*(train['풍속(m/s)']*3.6)**0.16 + 0.3965*(train['풍속(m/s)']*3.6)**0.16*train['기온(C)']
test['체감온도'] = 13.12 + 0.6215*test['기온(C)'] - 11.37*(test['풍속(m/s)']*3.6)**0.16 + 0.3965*(test['풍속(m/s)']*3.6)**0.16*test['기온(C)']

In [19]:
train.drop('풍속(m/s)', axis=1, inplace=True)
test.drop('풍속(m/s)', axis=1, inplace=True)

In [20]:
# train_x = train.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
# train_y = train['전력소비량(kWh)']

# test_x = test.drop(columns=['num_date_time', '일시'])

# X_train, X_test, y_train, y_test = train_test_split(train_x , train_y ,test_size=0.2, shuffle=True, random_state=42, stratify=train_x['건물번호'])
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [21]:
days = [18, 19, 20, 21, 22, 23, 24]

X_train = train[~((train['month'] == 8) & (train['day'].isin(days)))].drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
y_train = train[~((train['month'] == 8) & (train['day'].isin(days)))]['전력소비량(kWh)']

X_test = train[(train['month'] == 8) & (train['day'].isin(days))].drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
y_test = train[(train['month'] == 8) & (train['day'].isin(days))]['전력소비량(kWh)']


print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(185728, 21) (16800, 21) (185728,) (16800,)


In [22]:
#결측값을 0으로 채웁니다
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

## Regression Model Fit

In [23]:
#!pip install optuna
import optuna

In [24]:
def objective(trial):
    params = {
        "random_state": 42,
        "n_estimators": trial.suggest_int("n_estimators", 70, 120)
    }
    
    model = RandomForestRegressor(**params)
    model.fit(temp_X_train, temp_y_train)
    y_preds = model.predict(temp_X_test)
    return smape(temp_y_test, y_preds)

In [25]:
%%time 
metrics_type = pd.DataFrame()
for i in train['건물유형'].unique():
    temp_X_train = X_train[X_train['건물유형'] == i].drop(['건물유형'], axis=1)
    temp_y_train = y_train.loc[temp_X_train.index]
    temp_X_test = X_test[X_test['건물유형'] == i].drop(['건물유형'], axis=1)
    temp_y_test = y_test.loc[temp_X_test.index]
    
#     model = RandomForestRegressor(random_state=42)
#     model.fit(temp_X_train, temp_y_train)
#     y_preds = model.predict(temp_X_test)
#     temp = pd.DataFrame([i, smape(temp_y_test, y_preds)]).T
#     temp.columns = ['건물유형', '유형별_score']
    
    study = optuna.create_study()
    study.optimize(objective, n_trials=10)
    optuna.logging.set_verbosity(optuna.logging.CRITICAL)
    temp = pd.DataFrame([i, study.best_value, study.best_trial.params]).T
    temp.columns = ['건물유형', '유형별_score', '유형별_param']
    
    metrics_type = metrics_type.append(temp)
#     display(temp)
#     vis_feature_importances_(model, temp_X_train)    
    

[I 2023-08-25 21:54:50,419] A new study created in memory with name: no-name-1c64f9cd-4bc3-4d5c-a4a8-7c2758d4f817
[I 2023-08-25 21:55:03,432] Trial 0 finished with value: 6.980715928658862 and parameters: {'n_estimators': 95}. Best is trial 0 with value: 6.980715928658862.
[I 2023-08-25 21:55:14,815] Trial 1 finished with value: 6.9853562509754275 and parameters: {'n_estimators': 85}. Best is trial 0 with value: 6.980715928658862.
[I 2023-08-25 21:55:26,982] Trial 2 finished with value: 6.978666473519943 and parameters: {'n_estimators': 91}. Best is trial 2 with value: 6.978666473519943.
[I 2023-08-25 21:55:42,927] Trial 3 finished with value: 6.98452549266497 and parameters: {'n_estimators': 119}. Best is trial 2 with value: 6.978666473519943.
[I 2023-08-25 21:55:53,853] Trial 4 finished with value: 6.979698671578824 and parameters: {'n_estimators': 82}. Best is trial 2 with value: 6.978666473519943.
[I 2023-08-25 21:56:04,639] Trial 5 finished with value: 6.980746639670161 and parame

Wall time: 13min 51s


In [26]:
%%time 
metrics_bno = pd.DataFrame()
for i in train['건물번호'].unique():

    temp_X_train = X_train[X_train['건물번호'] == i].drop(['건물번호','건물유형'], axis=1)
    temp_y_train = y_train.loc[temp_X_train.index]
    temp_X_test = X_test[X_test['건물번호'] == i].drop(['건물번호','건물유형'], axis=1)
    temp_y_test = y_test.loc[temp_X_test.index]
        
#     model = RandomForestRegressor(random_state=42)
#     model.fit(temp_X_train, temp_y_train)
#     y_preds = model.predict(temp_X_test)
#     temp = pd.DataFrame([i, smape(temp_y_test, y_preds)]).T

    study = optuna.create_study()
    study.optimize(objective, n_trials=10)
    optuna.logging.set_verbosity(optuna.logging.CRITICAL)
    temp = pd.DataFrame([i, study.best_value, study.best_trial.params]).T
    temp.columns = ['건물번호', '건물별_score', '건물별_param']
    
    metrics_bno = metrics_bno.append(temp)
#     display(temp)
#     vis_feature_importances_(model, temp_X_train)    
#     print()

Wall time: 11min 58s


In [27]:
metrics_bno

,건물번호,건물별_score,건물별_param
0,1,6.216219,{'n_estimators': 86}
0,2,8.057845,{'n_estimators': 79}
0,3,9.011023,{'n_estimators': 120}
0,4,4.672384,{'n_estimators': 120}
0,5,6.015663,{'n_estimators': 107}
...,...,...,...
0,96,3.583813,{'n_estimators': 78}
0,97,6.744337,{'n_estimators': 70}
0,98,13.410299,{'n_estimators': 95}
0,99,4.172686,{'n_estimators': 109}


In [28]:
metrics_bno['건물번호'] = metrics_bno['건물번호'].astype(int)

In [29]:
metrics_bno = metrics_bno.merge(building_info[['건물유형','건물번호']], on=['건물번호'])

In [61]:
metrics_type

,건물유형,유형별_score,유형별_param
0,건물기타,6.967504,{'n_estimators': 106}
0,공공,5.609114,{'n_estimators': 100}
0,대학교,4.090848,{'n_estimators': 104}
0,데이터센터,0.571763,{'n_estimators': 101}
0,백화점및아울렛,4.823835,{'n_estimators': 113}
0,병원,3.916235,{'n_estimators': 92}
0,상용,5.999785,{'n_estimators': 86}
0,아파트,5.685256,{'n_estimators': 120}
0,연구소,4.465548,{'n_estimators': 115}
0,지식산업센터,3.226093,{'n_estimators': 102}


In [30]:
metrics = metrics_bno.merge(metrics_type, on=['건물유형'])
metrics[:2]

,건물번호,건물별_score,건물별_param,건물유형,유형별_score,유형별_param
0,1,6.216219,{'n_estimators': 86},건물기타,6.968193,{'n_estimators': 107}
1,2,8.057845,{'n_estimators': 79},건물기타,6.968193,{'n_estimators': 107}


In [31]:
# 유형보다 건물이 더 좋은 스코어를 가지는 건물번호 list
bno = metrics[metrics['건물별_score'] < metrics['유형별_score']]['건물번호'].values
len(bno), bno

(60,
 array([  1,   4,   5,   6,   8,   9,  11,  12,  13,  15,  16,  20,  22,
         23,  24,  25,  26,  27,  29,  32,  33,  35,  37,  38,  39,  41,
         44,  45,  48,  50,  52,  55,  56,  57,  58,  59,  61,  62,  64,
         68,  69,  70,  73,  75,  77,  79,  83,  84,  85,  86,  88,  89,
         90,  92,  93,  94,  96,  97,  99, 100]))

In [32]:
metrics_type['유형별_score'].mean(), metrics_bno['건물별_score'].mean()

(4.899693568826108, 5.200351151803313)

In [33]:
# 가중 평균 
metrics_type['유형별_score'].mean() * ((100-len(bno))/100) + metrics_bno['건물별_score'].mean() * (len(bno)/100)

5.080088118612431

In [34]:
# 5.102246917555636

In [35]:
metrics['score'] = metrics['유형별_score'] 
metrics.loc[metrics['건물번호'].isin(bno), 'score'] = metrics['건물별_score']

In [36]:
metrics['건물이낫다'] = metrics['건물별_score'] < metrics['유형별_score']

In [37]:
# metrics.to_excel('오차큰것부터_20230822-3.xlsx')

## Inference & Submit

In [65]:
metrics_type[:2]

,건물유형,유형별_score,유형별_param
0,건물기타,6.968193,{'n_estimators': 107}
0,공공,5.608233,{'n_estimators': 82}


In [109]:
%%time
result_type = pd.DataFrame()
for i in train['건물유형'].unique():
    
    temp_X_train = train[train['건물유형'] == i].drop(['건물유형','num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'], axis=1)
    temp_y_train = train.loc[temp_X_train.index][['전력소비량(kWh)']]
    
    model = RandomForestRegressor(n_estimators=metrics_type[metrics_type['건물유형'] == i]['유형별_param'][0]['n_estimators'], random_state=42)
    model.fit(temp_X_train, temp_y_train)
                                                 
    temp_test = test[test['건물유형'] == i].drop(['건물유형','num_date_time', '일시'], axis=1)
    y_preds = model.predict(temp_test)
    result_type = result_type.append( pd.concat([test[test['건물유형'] == i].reset_index(), pd.DataFrame(y_preds, columns=['answer_유형별']).reset_index()], axis=1) )
    
result_type.shape    

107
82
103
99
113
79
71
114
116
101
105
104
Wall time: 1min 37s


(16800, 26)

In [96]:
metrics_bno

,건물번호,건물별_score,건물별_param,건물유형
0,1,6.216219,{'n_estimators': 86},건물기타
1,2,8.057845,{'n_estimators': 79},건물기타
2,3,9.011023,{'n_estimators': 120},건물기타
3,4,4.672384,{'n_estimators': 120},건물기타
4,5,6.015663,{'n_estimators': 107},건물기타
...,...,...,...,...
95,96,3.583813,{'n_estimators': 78},호텔및리조트
96,97,6.744337,{'n_estimators': 70},호텔및리조트
97,98,13.410299,{'n_estimators': 95},호텔및리조트
98,99,4.172686,{'n_estimators': 109},호텔및리조트


In [108]:
%%time
result_bno = pd.DataFrame()
for i in train['건물번호'].unique():
    
    temp_X_train = train[train['건물번호'] == i].drop(['건물번호','건물유형','num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'], axis=1)
    temp_y_train = train.loc[temp_X_train.index][['전력소비량(kWh)']]
    
    model = RandomForestRegressor(n_estimators=metrics_bno[metrics_bno['건물번호'] == i]['건물별_param'].values[0]['n_estimators'],random_state=42)
    model.fit(temp_X_train, temp_y_train)
                                                 
    temp_test = test[test['건물번호'] == i].drop(['건물번호','건물유형','num_date_time', '일시'], axis=1)
    y_preds = model.predict(temp_test)
    result_bno = result_bno.append( pd.concat([test[test['건물번호'] == i].reset_index(), pd.DataFrame(y_preds, columns=['answer_건물별']).reset_index()], axis=1) )
    
result_bno.shape    

Wall time: 1min 18s


(16800, 26)

In [110]:
result = result_type[['num_date_time','건물번호','answer_유형별','weekday', 'time']].merge(result_bno[['num_date_time','건물번호','answer_건물별']], on=['num_date_time','건물번호'])
result['answer'] = result['answer_유형별'] 
result.loc[result['건물번호'].isin(bno), 'answer'] = result['answer_건물별']

# 후처리 
postprocessing = train.groupby(['건물번호', 'weekday','time'])['전력소비량(kWh)'].min().reset_index()
result = result.merge(postprocessing, on=['건물번호', 'weekday','time'])
result.loc[result['answer'] < result['전력소비량(kWh)'], 'answer'] = result['전력소비량(kWh)']

result[['num_date_time','answer']].to_csv('20230825-3_r.csv', index=False)